In [2]:
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import os

# Set LBP parameters
radius = 1
n_points = 8 * radius




In [3]:
# Define the function to extract LBP feature vector
def get_lbp_features(img):
    height, width = img.shape
    lbp = np.zeros((height-2*radius, width-2*radius), dtype=np.uint8)
    for i in range(radius, height-radius):
        for j in range(radius, width-radius):
            center = img[i,j]
            code = 0
            for k in range(n_points):
                x = i + int(radius * np.cos(2 * np.pi * k / n_points))
                y = j - int(radius * np.sin(2 * np.pi * k / n_points))
                if img[x,y] > center:
                    code += 1 << (n_points - 1 - k)
            lbp[i-radius,j-radius] = code
    feat_vector = lbp.flatten()
    return feat_vector

In [4]:
# Set the path to the FER2013 dataset folder
fer_folder = "C:\\mydata\\FE\\"
# Load the FER2013 dataset
fer_data = pd.read_csv(os.path.join(fer_folder, "fer2013.csv"))

In [12]:

# Extract the pixel values and emotion labels
X = []
y = []
for i in range(len(fer_data)):
    pixels = fer_data.loc[i, "pixels"].split(" ")
    pixels = np.array(pixels, dtype=np.uint8)
    pixels = pixels.reshape(48, 48)
    label = fer_data.loc[i, "emotion"]
    X.append(pixels)
    y.append(label)

# Extract LBP features from the images
X_lbp = []
for i in range(len(X)):
    lbp_feat = get_lbp_features(X[i])
    X_lbp.append(lbp_feat)

# Convert the features and labels arrays to numpy arrays
X_lbp = np.array(X_lbp)
y = np.array(y)


In [ ]:
import numpy as np

# Assume X_lbp and y have been computed

# Save X_lbp to file
np.save("C:\\mydata\\FE\\data\\X_lbp.npy", X_lbp)

# Save y to file
np.save("C:\\mydata\\FE\\data\\y.npy", y)



In [14]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_lbp, y, test_size=0.2, random_state=101)

Best Parameters: {'max_features': 'sqrt', 'n_estimators': 500}
Accuracy: 0.36772438813714814


In [17]:
# Save the trained model
import joblib
joblib.dump(CV_rfc.best_estimator_, "trained_modelRF.pkl")

['trained_modelRF.pkl']

In [20]:
# Load the trained model and predict the class of a single image
model = joblib.load("trained_modelRF.pkl")
img_path = "C:\\mydata\\JAFFE\\test1\\fear.37.jpg"
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
lbp_feat = get_lbp_features(img)
lbp_feat = lbp_feat.reshape(1, -1)
label = model.predict(lbp_feat)
print("Predicted Label:", label)

ValueError: X has 64516 features, but RandomForestClassifier is expecting 2116 features as input.